In [1]:
import cv2
import h5py
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc


from PIL import Image
from scipy import ndimage
from skimage import io, filters, morphology, segmentation

%matplotlib widget

In [2]:
import sys
sys.path = ['/Users/rejsmont/PycharmProjects/rdn-wdp-python/analysis'] + sys.path

In [3]:
basedir = '/Users/rejsmont/Google Drive/My Drive/Projects/RDN-WDP'

In [4]:
#rnaseq_file = basedir + '/contrib/scRNAseq/EyeAntennal_Combined_DG2_v4.csv'
#rnaseq = RNAseq3(rnaseq_file)
#aerts = rnaseq.data

#datadir = "/Users/rejsmont/Google Drive/My Drive/Projects/RDN-WDP/sharing/"
#counts = pd.read_csv(os.path.join(datadir, 'export', 'aerts_counts.csv'), index_col=0)
#counts.index = counts.index.astype(str)
#genes = counts.columns.to_frame().rename(columns={0: 'gene_ids'})
#cells = pd.read_csv(os.path.join(datadir, 'export', 'aerts_cells.csv'), index_col=0) \
#          .iloc[:, 0].str.split("-", n = 1, expand = True) \
#          .rename(columns={0: 'barcode', 1: 'replica'})
#cells.index = cells.index.astype(str)
#adata = sc.AnnData(X = counts.values, obs = cells, var=genes)
#aerts = adata.to_df()

In [5]:
aerts = pd.read_csv(os.path.join('r', 'seurat_count.csv')).T

In [6]:
ato_iJbqq8 = h5py.File(basedir + '/samples/999_ato_disc_1_iJbqq8.h5', 'r')

ato_mcherry = filters.gaussian(np.array(ato_iJbqq8['scaled/mCherry']), 3.0, preserve_range=True).astype(int)

ato_venus = np.array(ato_iJbqq8['scaled/Venus'])

ato_fish = io.imread('/Users/rejsmont/Desktop/ato smi-FISH/B_ato-mCherry-disc_1.tif')
ato_fish = ndimage.rotate(ato_fish, 130.5, (1, 2))
ato_fish = filters.gaussian(ato_fish[:,:,:,1], 1, preserve_range=True).astype(int)

ato_sa = ndimage.rotate(io.imread('/Users/rejsmont/Desktop/ato smi-FISH/atoSA-Gf7.tif'), 280)
ato_sa_mask = ato_sa[:, :, 0]
ato_sa_mask = filters.gaussian(ato_sa_mask, 3)
ato_sa_mask = ato_sa_mask > filters.threshold_isodata(ato_sa_mask)
ato_sa_mask = morphology.remove_small_objects(ato_sa_mask, 1500)
ato_sa_mask = filters.gaussian(ato_sa_mask, 10) > 0.25
ato_sa_mask = morphology.label(ato_sa_mask)

fig_1 = None

In [7]:
from CellModels.Clustering.IO import ClusteringReader

ato_p = ClusteringReader.read(basedir + '/processing/clustering/bigc100k6n20r1000_metadata.yml')
if ato_p is None:
    raise Exception
ato_t = ClusteringReader.read(basedir + '/clustering-h5/clustering_ato_dae5_s20r1f.h5')

In [8]:
from CellModels.Clustering.Figures import ClusterPlot
from CellModels.Cells.Figures import SamplePlot 
from matplotlib.ticker import MaxNLocator, FixedLocator
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as pe

def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

def plotgenes(ds, gene1, gene2, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    x = ds[gene1] + np.random.normal(0, 0.1, size=len(ds.index)).tolist()
    y = ds[gene2] + np.random.normal(0, 0.1, size=len(ds.index)).tolist()
    ax.scatter(x, y, alpha=0.1, c='k')
    ax.set_xlabel(str(gene1) + ' transcript count')
    ax.set_ylabel(str(gene2) + ' transcript count')
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_minor_locator(FixedLocator([0.5]))
    ax.xaxis.set_minor_locator(FixedLocator([0.5]))
    ax.yaxis.set_tick_params(labelleft=True)
    ax.grid(which='minor')
    n1 = len(ds[(ds[gene1] > 0)].index)
    n2 = len(ds[(ds[gene2] > 0)].index)
    n = len(ds[(ds[gene1] > 0) & (ds[gene2] > 0)].index)
    ax.text(0.95, 0.95, gene1 + ':' + str(n1) + '\n' + gene2 + ':' + str(n2) + '\n' + 'both:' + str(n),
        horizontalalignment='right', verticalalignment='top', transform = ax.transAxes)

class LinearClusterPlot(ClusterPlot):
    def _get_sc_params(self):
        return {
            'cmap': 'tab10',
            's': 40,
            'vmin': 0,
            'vmax': 1
        }

class LinearCherrySamplePlot(SamplePlot):
    def _get_sc_params(self):
        return {
            'cmap': 'gray',
            'vmin': 0.25,
            'vmax': 2,
            's': 40
        }

class LinearVenusSamplePlot(SamplePlot):
    def _get_sc_params(self):
        return {
            'cmap': 'gray',
            'vmin': 0,
            'vmax': 10,
            's': 40
        }
    
if fig_1 is not None:
    plt.close(fig_1)
fig_1 = plt.figure(figsize=cm2inch(24,24))

###
### eye disc scRNAseq
###
gs1 = gridspec.GridSpec(1, 3)
# ato vs elav
ax = fig_1.add_subplot(gs1[0, 0])
plotgenes(aerts, 'ato', 'elav', ax=ax)
ax.text(0.05, 0.95, 'A', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
# ato vs repo
ax = fig_1.add_subplot(gs1[0, 1])
plotgenes(aerts, 'ato', 'repo', ax=ax)
ax.text(0.05, 0.95, 'B', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
# ato vs da
ax = fig_1.add_subplot(gs1[0, 2])
plotgenes(aerts, 'ato', 'da', ax=ax)
ax.text(0.05, 0.95, 'C', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
gs1.tight_layout(fig_1, rect=[0, 0.67, 1, 1])

###
### eye disc imaging
###
gs2 = gridspec.GridSpec(1, 4, wspace=0, hspace=0)
# ato smi-FISH (Voxel size: 0.5687x0.5687x0.3462 micron^3)
ax = fig_1.add_subplot(gs2[0, 0])
ax.imshow(ato_fish[37], vmin=75, vmax=250, cmap='gray')
ax.set_ylim(550, 300)
ax.set_xlim(275, 525)
scalebar = AnchoredSizeBar(ax.transData, 25/0.5687, None, 'lower left', pad=1,
                           color='white', frameon=False, size_vertical=2/0.5687, label_top=True)
ax.add_artist(scalebar)
ax.text(0.05, 0.95, 'D', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()
# ato-mCherry (Voxel size: 0.1510x0.1510x0.1500 micrometer^3)
ax = fig_1.add_subplot(gs2[0, 1])
ax.imshow(ato_mcherry[50] / np.max(ato_mcherry[50]), vmax=0.2, cmap='gray')
ax.set_ylim(900, 100)
ax.set_xlim(700, 1500)
scalebar = AnchoredSizeBar(ax.transData, 25/0.1510, None, 'lower left', pad=1,
                           color='white', frameon=False, size_vertical=2/0.1510, label_top=True)
ax.add_artist(scalebar)
ax.text(0.05, 0.95, 'E', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()
# ato-T2A-Venus (Voxel size: 0.1510x0.1510x0.1500 micrometer^3)
ax = fig_1.add_subplot(gs2[0, 2])
ax.imshow(ato_venus[65] / np.max(ato_venus[70]), vmax=0.35, cmap='gray')
ax.set_ylim(900, 100)
ax.set_xlim(700, 1500)
scalebar = AnchoredSizeBar(ax.transData, 25/0.1510, None, 'lower left', pad=1,
                           color='white', frameon=False, size_vertical=2/0.1510, label_top=True)
ax.add_artist(scalebar)
ax.text(0.05, 0.95, 'F', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()
# ato S->A (Voxel size: 0.1328x0.1328x1 micron^3) - wrong scale - 1.75x underscan
ax = fig_1.add_subplot(gs2[0, 3])
ax.imshow(ato_sa[:, :, 1], cmap='gray', vmax=2000)
ax.set_ylim(875, 375)
ax.set_xlim(300, 800)
for i in range(1, ato_sa_mask.max() + 1):
    label = (ato_sa_mask == i).astype('uint8')
    points, _ = cv2.findContours(label, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    ax.fill(points[0][:,0,0], points[0][:,0,1], facecolor='none', edgecolor='white', ls='--')
scalebar = AnchoredSizeBar(ax.transData, 25/(0.1328*1.75), None, 'lower left', pad=1,
                           color='white', frameon=False, size_vertical=2/(0.1328*1.75), label_top=True)
ax.add_artist(scalebar)
ax.text(0.05, 0.95, 'G', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()
gs2.tight_layout(fig_1, rect=[0, .40, 1, .67], h_pad=0, w_pad=0)


gs3 = gridspec.GridSpec(1, 4, wspace=0, hspace=0)

ax = fig_1.add_subplot(gs3[0, 0])
LinearCherrySamplePlot(ato_p, 'p5dvda', ('Measurements', 'Normalized', 'mCherry')).plot_axes(ax)
ax.set_xlim(10, 35)
ax.set_ylim(20, -5)
ax.hlines(0, 10, 35, ls='dotted', color='r')
ax.text(0.05, 0.95, 'H', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()

ax = fig_1.add_subplot(gs3[0, 1])
LinearClusterPlot(ato_p, 'p5dvda', ('Cluster', 'ward', 6)).plot_axes(ax)
ax.set_xlim(10, 35)
ax.set_ylim(20, -5)
ax.hlines(0, 10, 35, ls='dotted', color='k')
ax.text(0.05, 0.95, 'I', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
ax.set_axis_off()

ax = fig_1.add_subplot(gs3[0, 2])
LinearVenusSamplePlot(ato_p, 'p5dvda', ('Measurements', 'Normalized', 'Venus')).plot_axes(ax)
ax.set_xlim(10, 35)
ax.set_ylim(20, -5)
ax.hlines(0, 10, 35, ls='dotted', color='r')
ax.text(0.05, 0.95, 'J', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='w', path_effects=[pe.Stroke(linewidth=5, foreground='k'), pe.Normal()])
ax.set_axis_off()

ax = fig_1.add_subplot(gs3[0, 3])
LinearClusterPlot(ato_t, 'p5dvda', ('Cluster', 'ward', 5)).plot_axes(ax)
ax.set_xlim(10, 35)
ax.set_ylim(20, -5)
ax.hlines(0, 10, 35, ls='dotted', color='k')
ax.text(0.05, 0.95, 'K', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
ax.set_axis_off()
gs3.tight_layout(fig_1, rect=[0, .13, 1, .40], h_pad=0.2, w_pad=0.2)


gs3.tight_layout(fig_1, rect=[0, .1575, 1, .4275], h_pad=0.2, w_pad=0.2, pad=1.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-8-9863c71151d6>:139: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  gs2.tight_layout(fig_1, rect=[0, .40, 1, .67], h_pad=0, w_pad=0)
<ipython-input-8-9863c71151d6>:179: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  gs3.tight_layout(fig_1, rect=[0, .13, 1, .40], h_pad=0.2, w_pad=0.2)
<ipython-input-8-9863c71151d6>:182: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  gs3.tight_layout(fig_1, rect=[0, .1575, 1, .4275], h_pad=0.2, w_pad=0.2, pad=1.2)


In [9]:
fig_1.savefig('fig_1.pdf')

In [10]:
ll = [1, None, 4, 5, 6]
lt = len(ll)
if ll:
    lx = [None] + [ll[i+1] - ll[i] if ll[i+1] is not None and ll[i] is not None else None for i in range(lt-1)]
else:
    lx = []
lx

[None, None, None, 1, 1]

In [11]:
ariss = pd.read_csv(os.path.join('/Users/rejsmont/Google Drive/My Drive/Projects/RDN-WDP/contrib/GSE115476_RAW',
                                 'GSM3178869_DMS.expr.txt'))
fig_2 = None

In [13]:
if fig_2 is not None:
    plt.close(fig_2)
fig_2 = plt.figure(figsize=cm2inch(24,8))

###
### eye disc scRNAseq
###
gs2 = gridspec.GridSpec(1, 3)
# ato vs elav
ax = fig_2.add_subplot(gs2[0, 0])
plotgenes(ariss, 'ato', 'elav', ax=ax)
ax.text(0.05, 0.95, 'A', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
# ato vs repo
ax = fig_2.add_subplot(gs2[0, 1])
plotgenes(ariss, 'ato', 'repo', ax=ax)
ax.text(0.05, 0.95, 'B', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
# ato vs da
ax = fig_2.add_subplot(gs2[0, 2])
plotgenes(ariss, 'ato', 'da', ax=ax)
ax.text(0.05, 0.95, 'C', size=24, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes,
        color='k', path_effects=[pe.Stroke(linewidth=5, foreground='w'), pe.Normal()])
gs1.tight_layout(fig_1, rect=[0, 0.67, 1, 1])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-13-9f85ae4880a7>:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  gs1.tight_layout(fig_1, rect=[0, 0.67, 1, 1])
